In [1]:
import os
import numpy as np
import mne

In [2]:
# set the participant number
PNUM = "01"

In [3]:
RAW_EOG_CHANNELS = [u'EXG1', u'EXG2', u'EXG3', u'EXG4']
MASTOID_CHANNELS = [u'EXG5', u'EXG6']

In [4]:
data_raw_file = os.path.join('raw_data', 'P' + PNUM + '-raw.fif')
raw = mne.io.read_raw_fif(data_raw_file)

Opening raw data file raw_data\P01-raw.fif...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 64)  idle
    Range : 0 ... 2478165 =      0.000 ...  4840.166 secs
Ready.


In [5]:
print(raw)
print(raw.info)

# plot eeg data
# raw.plot_psd(fmax=30)
# raw.plot(duration=5, n_channels=30)

<Raw | P01-raw.fif, 69 x 2478166 (4840.2 s), ~78 kB, data not loaded>
<Info | 11 non-empty values
 bads: 3 items (P8, P10, T8)
 ch_names: Fp1, AF7, AF3, F1, F3, F5, F7, FT7, FC5, FC3, FC1, C1, C3, C5, ...
 chs: 64 EEG, 4 EOG, 1 Stimulus
 custom_ref_applied: False
 file_id: 4 items (dict)
 highpass: 0.0 Hz
 lowpass: 104.0 Hz
 meas_date: 2015-01-28 17:39:57 UTC
 meas_id: 4 items (dict)
 nchan: 69
 projs: Average EEG reference: off
 sfreq: 512.0 Hz
>


In [6]:
# Drop bad channels - in place on raw
for bad_channel in raw.info['bads']:
    raw.drop_channels(bad_channel)
    print("dropped: " + bad_channel)

Removing projector <Projection | Average EEG reference, active : False, n_channels : 64>
dropped: P8
dropped: P10
dropped: T8


In [7]:
# mastoid channel stuff - according to openmirr proj
mne.io.set_eeg_reference(raw.load_data(), MASTOID_CHANNELS, copy=False) # inplace
raw.drop_channels(MASTOID_CHANNELS)

Reading 0 ... 2478165  =      0.000 ...  4840.166 secs...
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.


ValueError: Missing channels from ch_names required by include:
['EXG5', 'EXG6']

In [ ]:
RAW_COPY = raw.copy()

eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True, eog=False, stim=False, exclude=[])

# bandpass filter - keeping a frequency range between 0.5 (high pass filter) and 30 Hz (low pass filter)
filtered_data = raw.load_data().filter(0.5, 30, picks=eeg_picks)

### RUN BELOW CODE IF ICA NOT GENERATED

In [ ]:
# set up and fit the ICA - infomax method
ica = mne.preprocessing.ICA(method='infomax')
ica.fit(filtered_data)

In [ ]:
# auto-detect artifacts by simple statistics
ica.exclude = []
ica.detect_artifacts(filtered_data)
print(ica.exclude)

In [ ]:
# score EEG channels by EOG correlation
bad_comps = set()

for eog_channel in RAW_EOG_CHANNELS:
    bad, scores = ica.find_bads_eog(filtered_data, ch_name=eog_channel, l_freq=0.5, h_freq=30)
    ica.plot_scores(scores, exclude=bad, title='In Red, EOG artifact sources')
    bad_comps.update(bad)
    print(bad)
    print("=========================\n")

In [ ]:
# view ica components to remove based on EOG correl
ica.plot_components(picks=list(bad_comps), ch_type='eeg', title='', colorbar=True, show=True);

In [ ]:
# combine excluded components
ica.exclude.extend(list(bad_comps))
print(list(set(ica.exclude)))

In [ ]:
# save ICA result for later
ica.save('ica_data/P' + PNUM + '-ica.fif', overwrite=True)

### CONTINUE FROM HERE IF ICA GENERATED

In [ ]:
ica = mne.preprocessing.read_ica('ica_data/P' + PNUM + '-ica.fif')

In [ ]:
# apply the transformation
postica_data = ica.apply(filtered_data, exclude=ica.exclude)
#ica.plot_components(ch_type='eeg', title='', colorbar=True, show=True);
#ica.plot_properties(filtered_data, ch_type='eeg', title='', colorbar=True, show=True);